In [13]:
pip install openpyxl


     -------------------------------------- 250.0/250.0 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [137]:
import pandas as pd
import numpy as np
import os


In [117]:
# read dataset
# df = pd.read_excel('your_file.xlsx')
# Orderline dataset
df1 = pd.read_csv('Orderlijnen 01112022-30112022.csv', low_memory=False)

# Bundle Item data
df2 = pd.read_excel('Bundle_item_TE.xlsx')

# Item Overview data
df3 = pd.read_excel('Item_overview_TE.xlsx')




In [118]:
df1.head()

,Order_number,Initiator_code,Order_date,Order_date_time,Order_wms_entry_date,Processed_date,Erp_error,Ship_date,Preferred_ship_date,Delay,...,Reception_time,Shipping_way_name,Article_number,Product_number,Product_name_nl,Qty_original,Qty_actual,Vas_comment,Warehouse_name,Index
0,0080000341,HI,10/6/2022,11:26:02 AM,10/6/2022,10/6/2022,True,11/2/2022,10/6/2022,18.0,...,B. One hour before cut-off time,Bollore RO,AH047009YGR,AH047009YGR_HI,PULMOTIL AC 250MG/ML 240ML X1BOTL GR,420.0,420.0,NaN,Livlina Tessenderlo,2947180.0
1,0080000353,RC,10/26/2022,10:56:09 AM,10/27/2022,10/31/2022,True,11/2/2022,NaN,NaN,...,C. After cut-off time,ESSERS ROOM,MN432574XVV,MN432574XVV_RC,AGITA 10 WG 400G X1JAR VV,661.0,661.0,NaN,Livlina Tessenderlo,2958165.0
2,0080000366,RC,11/7/2022,9:04:13 AM,NaN,NaN,True,12/31/2999,NaN,NaN,...,C. After cut-off time,ESSERS ROOM,MN432574XVV,MN432574XVV_RC,AGITA 10 WG 400G X1JAR VV,648.0,0.0,NaN,Livlina Tessenderlo,3015097.0
3,0080000377,RC,11/15/2022,3:56:02 PM,11/18/2022,11/21/2022,True,11/23/2022,NaN,NaN,...,C. After cut-off time,ESSERS ROOM,AF742025KHUAF,AF742025KHUAF_RC,DENAGARD 10% PREMIX 25KG X1BAG HUAF,58.0,58.0,NaN,Livlina Tessenderlo,3170126.0
4,0080000377,RC,11/15/2022,3:56:02 PM,11/18/2022,11/21/2022,True,11/23/2022,NaN,NaN,...,C. After cut-off time,ESSERS ROOM,AF742025KHUAF,AF742025KHUAF_RC,DENAGARD 10% PREMIX 25KG X1BAG HUAF,60.0,60.0,NaN,Livlina Tessenderlo,3170122.0


In [119]:
df3.head()

,Host ID,Item No.,Catalogue Code,Description,Item Grp.,Strategy Code,Pickmethod,Environ. Code
0,2,0059873,81724004,NEUROBION AMPOULES - (6) BEL,GM,02_FRIGO,S - the item has to be lot tracked,F
1,2,0064212,(OUT)239276,(OUT)Oral B Tandenborstel Prothese,GEN,02_ALG,S - the item has to be lot tracked,B
2,2,0080804,239274,( Out ) Vicks Sinex Spray Nasal 15 ml,GEN,02_SU,S - the item has to be lot tracked,B
3,2,0083105,239281,( Out ) Fluodontyl Tandpasta 1350 50 ml,GM,02_ALG,S - the item has to be lot tracked,B
4,2,0083659,(OUT)239273,(OUT)Oral B Tandenborstel Plus 35 Orthodontic,GEN,02_ALG,S - the item has to be lot tracked,B


In [120]:
columns_to_copy = ["Order_date", "Nb_of_colli", "Nb_of_pallets",	"Article_number"	,"Product_number"	,"Qty_actual"]
df1_new = df1[columns_to_copy]
df1_new.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual
0,10/6/2022,0.0,1.0,AH047009YGR,AH047009YGR_HI,420.0
1,10/26/2022,0.0,1.0,MN432574XVV,MN432574XVV_RC,661.0
2,11/7/2022,0.0,0.0,MN432574XVV,MN432574XVV_RC,0.0
3,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,58.0
4,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,60.0


In [121]:
# Value to search for values
# Create a new column in df1 to store the extracted values
# df1_new['ChildQty'] = df1['Product_number'].map(df2.set_index('ProductCode')['ChildQty'])

# Use merge to match and append values from df2 to df1
merged_df = df1_new.merge(df2, left_on='Product_number', right_on='ProductCode', how='left')

merged_df.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,Host,HostArtikelCode,ProductCode,ProductNaam,ProductGroep,BundelType,ChildBundelType,Bundelbeschrijving,ChildQty,heigth,width,length,weight,Omgeving
0,10/6/2022,0.0,1.0,AH047009YGR,AH047009YGR_HI,420.0,HI,AH047009YGR,AH047009YGR_HI,PULMOTIL AC 250MG/ML 240ML X1BOTL GR,GMD Geneesmiddel,Full Case,Single Unit,FC/12 SU,12.0,145.0,197.0,260.0,3670.0,Kamertemperatuur (15-25°C)
1,10/26/2022,0.0,1.0,MN432574XVV,MN432574XVV_RC,661.0,RC,MN432574XVV,MN432574XVV_RC,AGITA 10 WG 400G X1JAR VV,PST Biocide,Full Case,Single Unit,FC/18 SU,18.0,235.0,362.0,363.0,9785.0,Kamertemperatuur (15-25°C)
2,11/7/2022,0.0,0.0,MN432574XVV,MN432574XVV_RC,0.0,RC,MN432574XVV,MN432574XVV_RC,AGITA 10 WG 400G X1JAR VV,PST Biocide,Full Case,Single Unit,FC/18 SU,18.0,235.0,362.0,363.0,9785.0,Kamertemperatuur (15-25°C)
3,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
drop_col = [6,7,8,9,10,11,12,13,15,16,17,18,19]

merged_df = merged_df.drop(merged_df.columns[drop_col], axis=1)
merged_df.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty
0,10/6/2022,0.0,1.0,AH047009YGR,AH047009YGR_HI,420.0,12.0
1,10/26/2022,0.0,1.0,MN432574XVV,MN432574XVV_RC,661.0,18.0
2,11/7/2022,0.0,0.0,MN432574XVV,MN432574XVV_RC,0.0,18.0
3,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,58.0,NaN
4,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,60.0,NaN


In [123]:
# Use merge to match and append values from df2 to df1
merged_df_new = merged_df.merge(df3, left_on='Product_number', right_on='Item No.', how='left')

merged_df_new.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Host ID,Item No.,Catalogue Code,Description,Item Grp.,Strategy Code,Pickmethod,Environ. Code
0,10/6/2022,0.0,1.0,AH047009YGR,AH047009YGR_HI,420.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/26/2022,0.0,1.0,MN432574XVV,MN432574XVV_RC,661.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/7/2022,0.0,0.0,MN432574XVV,MN432574XVV_RC,0.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
drop_col = [7,8,9,10,11,13]

merged_df_new = merged_df_new.drop(merged_df_new.columns[drop_col], axis=1)
merged_df_new.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code
0,10/6/2022,0.0,1.0,AH047009YGR,AH047009YGR_HI,420.0,12.0,NaN,NaN
1,10/26/2022,0.0,1.0,MN432574XVV,MN432574XVV_RC,661.0,18.0,NaN,NaN
2,11/7/2022,0.0,0.0,MN432574XVV,MN432574XVV_RC,0.0,18.0,NaN,NaN
3,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,58.0,NaN,NaN,NaN
4,11/15/2022,0.0,30.0,AF742025KHUAF,AF742025KHUAF_RC,60.0,NaN,NaN,NaN


In [125]:
# Full case, Single unit calculation and separation of automatic from manual

def full_case_calculation(df):
    # Calculation for the full case
    df['Full_case'] = np.where(df['ChildQty'].notna() & df['Qty_actual'].notna(),
                              df['Qty_actual'] // df['ChildQty'], 0)

    # Calculation for the single unit
    df['Single_unit'] = np.where(df['ChildQty'].notna() & df['Qty_actual'].notna(),
                                df['Qty_actual'] % df['ChildQty'], 0)

    # Set 'Single_unit' to 'Qty_actual' where 'Strategy Code' is NaN or not 'HB_OSR' or 'HB_OSR_TMB'
    df.loc[df['Strategy Code'].isna() | ~df['Strategy Code'].isin(["HB_OSR", "HB_OSR_TMB"]), 'Single_unit'] = df['Qty_actual']

    # Create the 'Automated' column based on 'Strategy Code'
    # 1 if Strategy Code is "HB_OSR", "HB_OSR_TMB"
    # 0 otherwise
    df['Automated'] = df['Strategy Code'].isin(["HB_OSR", "HB_OSR_TMB"]).astype(int)

    return df




In [126]:
first_result = full_case_calculation(merged_df_new)

# def update_single_unit(row):
#     if pd.isna(row['Strategy Code']) and row['Strategy Code'] not in ["HB_OSR", "HB_OSR_TMB"]:
#         return row['Qty_actual']
#     return row['Single_unit']

# first_result['Single_unit'] = first_result.apply(update_single_unit, axis=1)
first_result.tail(100)

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated
79677,11/28/2022,5.0,0.0,3181732,6104900,20.0,NaN,HB_OSR,B,0.0,0.0,1
79678,11/28/2022,5.0,0.0,2983799,1142250,0.0,NaN,HB_OSR,B,0.0,0.0,1
79679,11/28/2022,5.0,0.0,3181716,6104899,40.0,NaN,HB_OSR,B,0.0,0.0,1
79680,11/28/2022,5.0,0.0,3181773,6104902,60.0,NaN,HB_OSR,B,0.0,0.0,1
79681,11/28/2022,5.0,0.0,2542488,6000688,120.0,NaN,HB_OSR,B,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
79772,11/28/2022,1.0,0.0,3021508,6083748,6.0,NaN,OF_FRIGO,F,0.0,6.0,0
79773,11/30/2022,3.0,0.0,3021508,6083748,72.0,NaN,OF_FRIGO,F,0.0,72.0,0
79774,11/30/2022,1.0,0.0,3021508,6083748,6.0,NaN,OF_FRIGO,F,0.0,6.0,0
79775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0


In [130]:
# Some values in Item overview not in Bundle has NAN
# That is they dont have a full case therefore replace Single_unit with the values of Qty_actual

first_result.loc[first_result['ChildQty'].isna(), 'Single_unit'] = first_result['Qty_actual']

first_result.tail(20)

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated,Time1(hours)
79757,11/8/2022,1.0,0.0,3021508,6083748,12.0,NaN,OF_FRIGO,F,0.0,12.0,0,0.30
79758,11/9/2022,1.0,0.0,3021508,6083748,12.0,NaN,OF_FRIGO,F,0.0,12.0,0,0.30
79759,11/9/2022,1.0,0.0,3021508,6083748,20.0,NaN,OF_FRIGO,F,0.0,20.0,0,0.50
79760,11/11/2022,1.0,0.0,3021508,6083748,6.0,NaN,OF_FRIGO,F,0.0,6.0,0,0.15
79761,11/15/2022,1.0,0.0,3021508,6083748,12.0,NaN,OF_FRIGO,F,0.0,12.0,0,0.30
79762,11/17/2022,1.0,0.0,3021508,6083748,12.0,NaN,OF_FRIGO,F,0.0,12.0,0,0.30
79763,11/18/2022,2.0,0.0,3021508,6083748,36.0,NaN,OF_FRIGO,F,0.0,36.0,0,0.90
79764,11/18/2022,1.0,0.0,3021508,6083748,12.0,NaN,OF_FRIGO,F,0.0,12.0,0,0.30
79765,11/21/2022,1.0,0.0,3021508,6083748,6.0,NaN,OF_FRIGO,F,0.0,6.0,0,0.15
79766,11/22/2022,1.0,0.0,3021508,6083748,6.0,NaN,OF_FRIGO,F,0.0,6.0,0,0.15


In [129]:
# Calculate the workload for each date

def calculate_time(row):
    if row['Automated'] == 1:
        single_unit_time = (row['Single_unit'] * 60) / 200 / 60
        return single_unit_time
    else:
        x_i_time = (row['Full_case'] * 60) / 80 / 60
        y_i_time = (row['Single_unit'] * 60) / 40 / 60
        return x_i_time + y_i_time

# Apply the function to create a new 'Time' column
first_result['Time1(hours)'] = first_result.apply(calculate_time, axis=1)

first_result.tail()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated,Time1(hours)
79772,11/28/2022,1.0,0.0,3021508,6083748,6.0,NaN,OF_FRIGO,F,0.0,6.0,0,0.15
79773,11/30/2022,3.0,0.0,3021508,6083748,72.0,NaN,OF_FRIGO,F,0.0,72.0,0,1.80
79774,11/30/2022,1.0,0.0,3021508,6083748,6.0,NaN,OF_FRIGO,F,0.0,6.0,0,0.15
79775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0,NaN
79776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0,NaN


In [136]:
# Check specific date

# first_result[first_result['Order_date'] == "10/13/2022"]

In [133]:
# Aggregate the total number of hours by grouping the Order_date

# Convert the 'Date' column to a datetime object
# first_result['Order_date'] = pd.to_datetime(first_result['Order_date'], format='%d/%m/%Y')

# Group the data by date and calculate the sum of the 'Time' column
second_result = first_result.groupby('Order_date')['Time1(hours)'].sum().reset_index()
second_result

,Order_date,Time1(hours)
0,10/10/2022,0.000
1,10/11/2022,0.125
2,10/12/2022,2.720
3,10/13/2022,19.220
4,10/14/2022,4.045
...,...,...
123,9/5/2022,1.250
124,9/6/2022,2.850
125,9/7/2022,1.895
126,9/8/2022,1.385


In [134]:
sorted_result = second_result.sort_values(by='Order_date', ascending=False)
sorted_result

,Order_date,Time1(hours)
127,9/9/2022,3.100
126,9/8/2022,1.385
125,9/7/2022,1.895
124,9/6/2022,2.850
123,9/5/2022,1.250
...,...,...
4,10/14/2022,4.045
3,10/13/2022,19.220
2,10/12/2022,2.720
1,10/11/2022,0.125


In [140]:
# Merge all the excel files

dataframes = []

directory = 'C:\\Users\\harve\\Downloads\\Orderline_dataset\\'

for filename in os.listdir(directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"): 
        filepath = os.path.join(directory, filename)
        df = pd.read_excel(filepath)
        dataframes.append(df)


# Merge the file
merged_df = pd.concat(dataframes, ignore_index=True)


# export the dataframe to excel file

# merged_df.to_excel('merged_file.xlsx', index=False)


In [141]:
merged_df.head()

,Order_number,Initiator_code,Order_date,Order_date_time,Order_wms_entry_date,Processed_date,Erp_error,Ship_date,Preferred_ship_date,Delay,...,Pharmacie des Plantes sa,13:00:00,B. One hour before cut-off time,BPSRO,2192938,1142264,Steovit Forte Sinaas1000mg/880IE - 30 Bruistabl,6,6.1,1173873
0,01000100,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
# Dimension of the dataset

merged_df.shape

(1307820, 75)

In [146]:
columns_to_copy = ["Order_date", "Nb_of_colli", "Nb_of_pallets",	"Article_number"	,"Product_number"	,"Qty_actual"]
real_df1 = merged_df[columns_to_copy]
real_df1.head()


,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual
0,2022-01-04,20.0,0.0,4380564,5407009330063,800.0
1,2022-01-04,1.0,0.0,4393997,5407009330391,6.0
2,2022-01-04,1.0,0.0,4394060,5407009330469,6.0
3,2022-01-04,1.0,0.0,4394094,5407009330490,6.0
4,2022-01-04,1.0,0.0,4380564,5407009330063,12.0


In [147]:
# Use merge to match and append values from df2 to df1
real_df2 = real_df1.merge(df2, left_on='Product_number', right_on='ProductCode', how='left')

real_df2.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,Host,HostArtikelCode,ProductCode,ProductNaam,ProductGroep,BundelType,ChildBundelType,Bundelbeschrijving,ChildQty,heigth,width,length,weight,Omgeving
0,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
drop_col = [6,7,8,9,10,11,12,13,15,16,17,18,19]

real_df2 = real_df2.drop(real_df2.columns[drop_col], axis=1)
real_df2.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty
0,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN
1,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN
2,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN
3,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN
4,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN


In [149]:
# Use merge to match and append values from df2 to df1
real_df3 = real_df2.merge(df3, left_on='Product_number', right_on='Item No.', how='left')

real_df3.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Host ID,Item No.,Catalogue Code,Description,Item Grp.,Strategy Code,Pickmethod,Environ. Code
0,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
drop_col = [7,8,9,10,11,13]

real_df3 = real_df3.drop(real_df3.columns[drop_col], axis=1)
real_df3.head()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code
0,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN
1,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN
2,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN
3,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN
4,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN


In [151]:
first_result1 = full_case_calculation(real_df3)

first_result1.tail(100)

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated
1343994,2022-12-26,25.0,0.0,2562585,6001663,0.0,NaN,HB_OSR,B,0.0,0.0,1
1343995,2022-12-26,25.0,0.0,0072298,1142176,20.0,NaN,HB_OSR,B,0.0,0.0,1
1343996,2022-12-26,25.0,0.0,0083725,1142236,20.0,NaN,HB_MTE,B,0.0,20.0,0
1343997,2022-12-26,25.0,0.0,3181831,6104905,20.0,NaN,HB_OSR,B,0.0,0.0,1
1343998,2022-12-26,25.0,0.0,0491290,1142101,0.0,NaN,HB_OSR,B,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1344089,2022-12-16,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344090,2022-12-20,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344091,2022-12-27,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344092,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0


In [152]:
first_result1.loc[first_result1['ChildQty'].isna(), 'Single_unit'] = first_result1['Qty_actual']

first_result1.tail(20)

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated
1344074,2022-12-08,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344075,2022-12-09,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344076,2022-12-09,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344077,2022-12-14,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344078,2022-12-14,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344079,2022-12-14,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344080,2022-12-14,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344081,2022-12-15,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344082,2022-12-15,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344083,2022-12-15,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0


In [153]:
# Apply the function to create a new 'Time' column
first_result1['Time1(hours)'] = first_result1.apply(calculate_time, axis=1)

first_result1.tail()

,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated,Time1(hours)
1344089,2022-12-16,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0,0.0
1344090,2022-12-20,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0,0.0
1344091,2022-12-27,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0,0.0
1344092,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0,NaN
1344093,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0,NaN


In [154]:
second_result1 = first_result1.groupby('Order_date')['Time1(hours)'].sum().reset_index()
second_result1


,Order_date,Time1(hours)
0,2021-01-27,0.4250
1,2021-03-11,2.2500
2,2022-01-01,0.4650
3,2022-01-03,2975.7100
4,2022-01-04,869.6200
...,...,...
472,2023-07-26,1136.5475
473,2023-07-27,1079.3550
474,2023-07-28,1243.4350
475,2023-07-29,2.8600


In [156]:
sorted_result1 = second_result1.sort_values(by='Order_date', ascending=True)
sorted_result1

,Order_date,Time1(hours)
0,2021-01-27,0.4250
1,2021-03-11,2.2500
2,2022-01-01,0.4650
3,2022-01-03,2975.7100
4,2022-01-04,869.6200
...,...,...
472,2023-07-26,1136.5475
473,2023-07-27,1079.3550
474,2023-07-28,1243.4350
475,2023-07-29,2.8600


## Outbound Data Repair

In [180]:
# Convert the 'Date' column to a datetime object
merged_df['Order_date'] = pd.to_datetime(merged_df['Order_date']).dt.strftime('%d-%m-%Y')

In [181]:
merged_df.head()

,Order_number,Initiator_code,Order_date,Order_date_time,Order_wms_entry_date,Processed_date,Erp_error,Ship_date,Preferred_ship_date,Delay,...,Pharmacie des Plantes sa,13:00:00,B. One hour before cut-off time,BPSRO,2192938,1142264,Steovit Forte Sinaas1000mg/880IE - 30 Bruistabl,6,6.1,1173873
0,01000100,KP,04-01-2022,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01000101,KP,04-01-2022,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01000101,KP,04-01-2022,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01000101,KP,04-01-2022,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01000101,KP,04-01-2022,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
# Group date by Colli
new_colli = merged_df.groupby(['Order_number', 'Order_date', 'Nb_of_colli']).size().reset_index(name='count')

unique_order_number_colli = new_colli.drop_duplicates(subset=['Order_number'])
# new_colli.head()

# Group date by Palette
new_palette = merged_df.groupby(['Order_number', 'Order_date', 'Nb_of_pallets']).size().reset_index(name='count')

unique_order_number_pal = unique_order_numbers = new_palette.drop_duplicates(subset=['Order_number'])


In [207]:
# Check for duplicate order numbers in Palette and Colli
are_order_nums_unique = not unique_order_number_pal['Order_number'].duplicated().any()

if are_order_nums_unique:
    print("All order numbers are unique.")
else:
    print("There are duplicate order numbers.")

All order numbers are unique.


In [208]:
new_drop = [0,3]

# unique_order_number_pal = unique_order_number_pal[unique_order_number_pal['Nb_of_pallets'] != 0].drop(unique_order_number_pal.columns[new_drop], axis=1)
unique_order_number_pal = unique_order_number_pal.drop(unique_order_number_pal.columns[new_drop], axis=1)

# unique_order_number_colli = unique_order_number_colli[unique_order_number_colli['Nb_of_colli'] != 0].drop(unique_order_number_colli.columns[new_drop], axis=1)
unique_order_number_colli = unique_order_number_colli.drop(unique_order_number_colli.columns[new_drop], axis=1)

In [209]:
print(f'The dimension of the Colli dataset is {unique_order_number_colli.shape}')
print(f'The dimension of the Palette dataset is {unique_order_number_pal.shape}')

The dimension of the Colli dataset is (273173, 2)
The dimension of the Palette dataset is (273173, 2)


In [210]:
unique_order_number_pal = unique_order_number_pal.groupby('Order_date')['Nb_of_pallets'].sum().reset_index()

unique_order_number_colli = unique_order_number_colli.groupby('Order_date')['Nb_of_colli'].sum().reset_index()

In [217]:
unique_order_number_pal.sort_values(by='Order_date', ascending=True)

,Order_date,Nb_of_pallets
0,01-01-2022,0.0
1,01-02-2023,138.0
2,01-03-2022,0.0
3,01-03-2023,46.0
4,01-04-2022,0.0
...,...,...
472,31-05-2023,498.0
473,31-07-2023,18.0
474,31-08-2022,226.0
475,31-10-2022,73.0
